## Part 1
Given your actual map, after `10000` bursts of activity, **how many bursts cause a node to become infected**? Do not count nodes that begin infected.

### Grid class

In [1]:
class Grid(object):
    def __init__(self, map_):
        self._grid = {}
        i, j_start = -(len(map_) // 2), -(len(map_[0]) // 2)
        for row in map_:
            j = j_start
            for c in row:
                self.set_node(c, (i, j))
                j += 1
            i += 1
        self._direction = (-1, 0)  # Up
        self._position = (0, 0)
        self._bursts, self._got_infected = 0, 0

    def get_node(self, position=None):
        if position is None:
            position = self._position
        if position not in self._grid:
            self._grid[position] = '.'
        return self._grid[position]

    def set_node(self, state, position=None):
        if position is None:
            position = self._position
        self._grid[position] = state

    DIR_STR = {(-1, 0): '↑', (0, 1): '>', (1, 0): '↓', (0, -1): '<'}

    def __str__(self):
        i_coords = set([pos[0] for pos in self._grid.keys()])
        j_coords = set([pos[1] for pos in self._grid.keys()])
        si, ei, sj, ej = min(i_coords), max(i_coords), min(j_coords), max(
            j_coords)
        pprint = []
        for i in range(si, ei + 1):
            row = []
            for j in range(sj, ej + 1):
                pos = (i, j)
                show_arrow = pos == self._position
                d_char = Grid.DIR_STR[self._direction] if show_arrow else ' '
                char = self.get_node(pos)
                row.append(f'{d_char}{char}{d_char}')
            pprint.append(''.join(row))
        return f'After burst #{self._bursts:02}, {self._got_infected} got infected.\n' + '\n'.join(
            pprint)

    R_TURNS = {(-1, 0): (0, 1), (0, 1): (1, 0), (1, 0): (0, -1), (0, -1): (-1, 0)}

    def turn_r(self):
        self._direction = self.R_TURNS[self._direction]

    L_TURNS = {value: key for key, value in R_TURNS.items()}

    def turn_l(self):
        self._direction = self.L_TURNS[self._direction]

    def move(self):
        self._bursts += 1
        self._position = (self._position[0] + self._direction[0],
                          self._position[1] + self._direction[1])

    def burst(self, iterations=1, debug=False):
        for i in range(iterations):
            if self.get_node() == '.':  # Clean
                self.turn_l()
                self.set_infected()
            else:  # Infected
                self.turn_r()
                self.set_clean()
            self.move()
            if debug:
                print(self)

    def set_infected(self):
        self.set_node('#')
        self._got_infected += 1

    def set_clean(self):
        self.set_node('.')

### Testing

In [2]:
def assert_grid(grid, got_infected):
    message = f'Expected got infected = {got_infected}, got {grid._got_infected}'
    assert grid._got_infected == got_infected, message

test_in = '..#/#../...'.split('/')
grid = Grid(test_in)
grid.burst(7)
print(grid)
assert_grid(grid, 5)

grid.burst(70 - 7)
print(grid)
assert_grid(grid, 41)

After burst #07, 5 got infected.
 #  . >.> # 
 #  #  #  . 
 .  .  .  . 
After burst #70, 41 got infected.
 .  .  .  #  #  .  . 
 .  .  #  .  .  #  . 
 .  #  .  .  .  .  # 
 #  .  # ↑.↑ .  .  # 
 #  .  #  .  .  #  . 
 .  .  .  #  #  .  . 


### Processing puzzle input

In [3]:
puzzle_in = [line[:-1] for line in open('in/day22.txt', 'r')]
grid = Grid(puzzle_in)
grid.burst(10000)
print(f'Part 1 answer is: {grid._got_infected} got infected after 10K bursts')
assert_grid(grid, 5433)

Part 1 answer is: 5433 got infected after 10K bursts


## Part 2 - Evolved virus

### Evolved class

In [4]:
class Evolved(Grid):
    REV_TURNS = {
        (-1, 0): (1, 0),
        (0, 1): (0, -1),
        (1, 0): (-1, 0),
        (0, -1): (0, 1)
    }

    def turn_reverse(self):
        self._direction = self.REV_TURNS[self._direction]

    NEXT_STATES = {'.': 'W', 'W': '#', '#': 'F', 'F': '.'}

    def burst(self, iterations=1, debug=False):
        for i in range(iterations):
            state = self.get_node()
            if state == '.':  # Clean
                super(Evolved, self).turn_l()
                self.set_node('W')
            elif state == 'W':  # Weakened
                self.set_infected()
            elif state == '#':  # Infected
                super(Evolved, self).turn_r()
                self.set_node('F')
            elif state == 'F':  # Flagged
                self.turn_reverse()
                self.set_clean()

            self.move()
            self.get_node()
            if debug:
                print(self)

### Testing

In [5]:
evolved = Evolved(test_in)
evolved.burst(7)
evolved.burst(100-7)
assert_grid(evolved, 26)

evolved = Evolved(test_in)
evolved.burst(10000000)
assert_grid(evolved, 2511944)

### Processing puzzle input

In [6]:
evolved = Evolved(puzzle_in)
evolved.burst(10000000)
print(f'Part 2 answer is: {evolved._got_infected} got infected after 10M bursts')
assert_grid(evolved, 2512599)

Part 2 answer is: 2512599 got infected after 10M bursts
